https://github.com/Moon-xm/HMM_NER

#  人名PER, 地名LOC, 机构名ORG, 其他O

In [3]:
import numpy as np
from tqdm import tqdm

In [4]:
class HMM_model:
    def __init__(self):
        self.n_tag = 7  # 表示所有标签个数
        self.n_char = 65535  # 所有字符的Unicode编码个数
        self.epsilon = 1e-100  # 无穷小量
        self.tag2idx = {'B-PER': 0,
                        'I-PER': 1,
                        'B-LOC': 2,
                        'I-LOC': 3,
                        'B-ORG': 4,
                        'I-ORG': 5,
                        'O': 6}
        self.idx2tag = dict(zip(self.tag2idx.values(), self.tag2idx.keys()))
        self.A = np.zeros((self.n_tag, self.n_tag))  # 转移概率矩阵,shape:7*7
        self.B = np.zeros((self.n_tag, self.n_char))  # 发射概率矩阵,shape:7*字的个数
        self.pi = np.zeros(self.n_tag)  # 初始隐状态概率,shape：4

    def train(self, corpus_path):
        """函数说明： 训练HMM模型, 得到模型参数pi,A,B"""
        with open(corpus_path, mode='r', encoding='utf-8') as fr:
            lines = fr.readlines()
        print('开始训练数据：')
        for i in tqdm(range(len(lines))):
            if len(lines[i]) == 1:
                continue
            else:
                cur_char, cur_tag = lines[i].split()
                self.B[self.tag2idx[cur_tag]][ord(cur_char)] += 1
                if len(lines[i - 1]) == 1:
                    self.pi[self.tag2idx[cur_tag]] += 1
                    continue
                pre_char, pre_tag = lines[i - 1].split()
                self.A[self.tag2idx[pre_tag]][self.tag2idx[cur_tag]] += 1
        self.pi[self.pi == 0] = self.epsilon  # 防止数据下溢,对数据进行对数归一化
        self.pi = np.log(self.pi) - np.log(np.sum(self.pi))
        self.A[self.A == 0] = self.epsilon
        self.A = np.log(self.A) - np.log(np.sum(self.A, axis=1, keepdims=True))
        self.B[self.B == 0] = self.epsilon
        self.B = np.log(self.B) - np.log(np.sum(self.B, axis=1, keepdims=True))
        np.savetxt('pi', self.pi)
        np.savetxt('A', self.A)
        np.savetxt('B', self.B)
        print('训练完毕！')

    def viterbi(self, Obs):
        """
        函数说明： 使用viterbi算法进行解码
        Parameter：Obs - 要解码的文本string
        Return:path - 最可能的隐状态路径

        """
        T = len(Obs)
        delta = np.zeros((T, self.n_tag))  # shape: 观测文本数量*7
        psi = np.zeros((T, self.n_tag))  # shape: 观测文本数量*7
        delta[0] = self.pi[:] + self.B[:, ord(Obs[0])]  # 初始化
        for i in range(1, T):
            temp = delta[i - 1].reshape(self.n_tag, -1) + self.A  # 这里运用到了矩阵的广播算法
            delta[i] = np.max(temp, axis=0)
            delta[i] = delta[i, :] + self.B[:, ord(Obs[i])]
            psi[i] = np.argmax(temp, axis=0)
        path = np.zeros(T)
        path[T - 1] = np.argmax(delta[T - 1])
        for i in range(T - 2, -1, -1):  # 回溯
            path[i] = int(psi[i + 1][int(path[i + 1])])
        return path

    def predict(self, Obs):
        T = len(Obs)
        path = self.viterbi(Obs)
        for i in range(T):
            print(Obs[i]+self.idx2tag[path[i]]+'_|', end='')


In [9]:
model = HMM_model()
model.train('BIO_train.txt')

100%|████████████████████████████████████████████████████████████████████████| 66769/66769 [00:00<00:00, 382563.49it/s]

开始训练数据：


训练完毕！


In [13]:
s = '林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？'        
model.predict(s)

林B-PER_|徽I-PER_|因O_|什O_|么O_|理O_|由O_|拒O_|绝O_|了O_|徐O_|志O_|摩B-LOC_|而O_|选O_|择O_|梁B-PER_|思I-PER_|成I-PER_|为O_|终O_|身O_|伴O_|侣O_|？O_|

In [12]:
model.predict('谢娜为李浩菲澄清网络谣言，之后她的两个行为给自己加分')

谢O_|娜O_|为O_|李O_|浩O_|菲O_|澄B-PER_|清I-PER_|网O_|络O_|谣O_|言O_|，O_|之O_|后O_|她O_|的O_|两O_|个O_|行O_|为O_|给O_|自O_|己O_|加O_|分O_|

In [10]:
model.predict("张三是河北省保定市河北大学网络空间安全与计算机学院的一名学生")

张O_|三O_|是O_|河B-LOC_|北I-LOC_|省I-LOC_|保B-LOC_|定I-LOC_|市I-LOC_|河I-LOC_|北I-LOC_|大I-LOC_|学O_|网O_|络O_|空O_|间O_|安O_|全O_|与O_|计O_|算O_|机O_|学O_|院O_|的O_|一O_|名O_|学O_|生O_|

In [11]:
model.predict("张明是河北大学的一名学生")

张B-PER_|明I-PER_|是O_|河B-ORG_|北I-ORG_|大I-ORG_|学I-ORG_|的O_|一O_|名O_|学O_|生O_|